<a href="https://colab.research.google.com/github/developerY/MojoMax/blob/main/MojoMax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install modular --index-url https://dl.modular.com/public/nightly/python/simple/ --extra-index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://dl.modular.com/public/nightly/python/simple/, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.2/146.2 kB 4.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-http to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-exporter-prometheus to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.5/95.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.8 MB/s e

In [1]:
!nvidia-smi

Thu Aug 28 03:14:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import numpy as np
from max.driver import CPU, Accelerator, Tensor, accelerator_count
from max.dtype import DType
from max.engine import InferenceSession
from max.graph import DeviceRef, Graph, TensorType, ops

In [3]:
accelerator_count()

1

In [4]:
device = CPU() if accelerator_count() == 0 else Accelerator()
device

Device(type=gpu,id=0)

In [5]:
vector_width = 10
dtype = DType.float32

with Graph(
    "vector_addition",
    input_types=[
        TensorType(
            dtype,
            shape=[vector_width],
            device=DeviceRef.from_device(device),
        ),
        TensorType(
            dtype,
            shape=[vector_width],
            device=DeviceRef.from_device(device),
        ),
    ],
) as graph:
    lhs, rhs = graph.inputs
    output = lhs + rhs
    graph.output(output)

In [6]:
session = InferenceSession(
    devices=[device],
)

model = session.load(graph)

In [7]:
lhs_values = np.random.uniform(size=(vector_width)).astype(np.float32)
rhs_values = np.random.uniform(size=(vector_width)).astype(np.float32)

lhs_tensor = Tensor.from_numpy(lhs_values).to(device)
rhs_tensor = Tensor.from_numpy(rhs_values).to(device)

In [8]:
result = model.execute(lhs_tensor, rhs_tensor)[0]

result = result.to(CPU())

In [9]:
print("Left-hand-side values:")
print(lhs_values)
print()

print("Right-hand-side values:")
print(rhs_values)
print()

print("Graph result:")
print(result.to_numpy())
print()

print("Expected result:")
print(lhs_values + rhs_values)

Left-hand-side values:
[0.78326505 0.80520815 0.7258589  0.30281925 0.35992864 0.77140385
 0.43595538 0.46135175 0.95262194 0.08872832]

Right-hand-side values:
[0.17332724 0.33674908 0.1111757  0.11207277 0.36297217 0.8505733
 0.30538553 0.22039181 0.6334334  0.1559453 ]

Graph result:
[0.9565923  1.1419573  0.83703464 0.41489202 0.7229008  1.6219771
 0.7413409  0.68174356 1.5860553  0.24467361]

Expected result:
[0.9565923  1.1419573  0.83703464 0.41489202 0.7229008  1.6219771
 0.7413409  0.68174356 1.5860553  0.24467361]


In [10]:
from max.entrypoints.llm import LLM
from max.pipelines import PipelineConfig
from max.serve.config import Settings

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [11]:
model_path = "Qwen/Qwen2.5-0.5B-Instruct"
print(f"Loading model: {model_path}")
pipeline_config = PipelineConfig(model_path=model_path)
settings = Settings()
llm = LLM(settings, pipeline_config)

prompts = [
    "The fastest way to learn python is",
]

print("Generating responses...")
responses = llm.generate(prompts, max_new_tokens=50)

for i, (prompt, response) in enumerate(zip(prompts, responses)):
    print(f"========== Response {i} ==========")
    print(prompt + response)
    print()

Loading model: Qwen/Qwen2.5-0.5B-Instruct


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

Exception ignored in: <function LLM.__del__ at 0x7aadc5fdc180>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/max/entrypoints/llm.py", line 89, in __del__
    self._pc.set_canceled()
    ^^^^^^^^
AttributeError: 'LLM' object has no attribute '_pc'


TypeError: LLM.__init__() takes 2 positional arguments but 3 were given